In [1]:
import os
import ast
import torch
from transformers import AutoModelForCTC, Wav2Vec2Processor
import numpy as np
import pandas as pd
import soundfile as sf
from src.text_processing import *
from src.data_processing import *
from src.ui_tools import *
from src.audio_processing import *

# Add espeak's shared library directory
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/lib'

In [2]:
# Load the cleaned data
data_path = 'data/df_test_cleaned.csv'
tests_df = pd.read_csv(data_path)

# We only keep the rows where the testType is readingTestFluencE
readingTestFluencE_df = tests_df[tests_df['testType'] == 'readingTestFluencE']

# Apply conversion functions to testResults and evaluationResults columns
readingTestFluencE_df['testResults'] = readingTestFluencE_df['testResults'].apply(lambda x: convert_str_to_dct_eval(x))
readingTestFluencE_df['evaluationResults'] = readingTestFluencE_df['evaluationResults'].apply(lambda x: convert_str_to_dct_eval(x))

/var/folders/kn/xrv1vh8j0k99dnh2mwl662xc0000gn/T/ipykernel_10543/537599615.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readingTestFluencE_df['testResults'] = readingTestFluencE_df['testResults'].apply(lambda x: convert_str_to_dct_eval(x))
/var/folders/kn/xrv1vh8j0k99dnh2mwl662xc0000gn/T/ipykernel_10543/537599615.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readingTestFluencE_df['evaluationResults'] = readingTestFluencE_df['evaluationResults'].apply(lambda x: convert_str_to_dct_eval(x))


In [3]:
# We extract the audio files and convert them to .wav format
save_recordings_as_wav(readingTestFluencE_df, output_dir='wav_files/readingTestFluencE', target_sample_rate=16000, channels=1)

In [ ]:
# The tests id of the 10 selected readingTestFluencE tests
tests_id = [
    '2BB671AA-2F6A-4346-8B76-F0C89C236390',
    '3B545E56-D802-4380-9993-21C11066B12E',
    '5C1C826F-E778-48C3-9170-6BF943175984',
    '046E4FEB-E284-48D5-922E-616DA7651F02',
    '75A80925-F8CF-463D-AFED-5CC399848CC2',
    '102DCD09-43EA-434D-A590-0FA5C7C7C1B3',
    '098522E8-2203-425E-85E5-5809D5B0B523',
    '79055215-1979-42D3-9B26-B9C6DD935D83',
    'ABD81BE7-7629-4816-8241-7ECBF32DFFFA',
]

In [4]:
# We extract the row with id = 75A80925-F8CF-463D-AFED-5CC399848CC2
test_id = '75A80925-F8CF-463D-AFED-5CC399848CC2'
test_row = readingTestFluencE_df[readingTestFluencE_df['id'] == test_id]

# Extract recordings and their corresponding evaluation results (e.g., 'wordsState')
recording = test_row['testResults'].apply(
    lambda x: x['recording'] if 'recording' in x else None).dropna().tolist()

evaluation_result = test_row['evaluationResults'].apply(
    lambda x: x['wordsState'] if 'wordsState' in x else None).dropna().tolist()

# Create the interactive audio player with evaluation results
create_audio_player_with_results(recording, evaluation_result)

Output()

Output()

In [ ]:
# If the transcription file already exists, we do nothing
transcription_file = 'testPhoneme_deletion_transcriptions.csv'
file_exists = os.path.isfile(transcription_file)

if not(file_exists):
    # We extract the text that is read by the children during the readingTestFluencE tests
    test = readingTestFluencE_df[readingTestFluencE_df['id'] == tests_id[0]]['testParameters'].values[0]
    test_dict = ast.literal_eval(test)
    selected_text = test_dict['textSelected']['text']

    save_phonetic_transcription_to_csv(selected_text, test_type='readingTestFluencE', folder='transcriptions', file_name='readingTestFluencE_transcriptions.csv')

Phonetic transcription saved to transcriptions/readingTestFluencE_transcriptions.csv


In [6]:
# We only keep the rows where the id is in the list of tests_id
readingTests = readingTestFluencE_df[readingTestFluencE_df['id'].apply(lambda x: x in tests_id)]

In [7]:
# Extract recordings and their corresponding evaluation results (e.g., 'wordsState')
recordings = readingTests['testResults'].apply(
    lambda x: x['recording'] if 'recording' in x else None).dropna().tolist()

evaluation_results = readingTests['evaluationResults'].apply(
    lambda x: x['wordsState'] if 'wordsState' in x else None).dropna().tolist()

# Create the interactive audio player with evaluation results
create_audio_player_with_results(recordings, evaluation_results)

Output()

Output()

In [8]:
# Initialize the model and processor
MODEL_ID = "Cnam-LMSSC/wav2vec2-french-phonemizer"
model = AutoModelForCTC.from_pretrained(MODEL_ID)
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)

In [9]:
# Folder containing the audio files
audio_folder = 'wav_files/readingTestFluencE/'

# List all the .wav files in the folder
audio_files = [f for f in os.listdir(audio_folder) if f.endswith('.wav')]

# Initialize an empty list to store the results
transcriptions = []

# Process each file
for audio_file in audio_files:
    # Load the audio file
    audio_path = os.path.join(audio_folder, audio_file)
    audio, _ = sf.read(audio_path)
    
    # Preprocess the audio and prepare the inputs for the model
    inputs = processor(np.array(audio), sampling_rate=16_000., return_tensors="pt")
    
    # Get the model's predictions
    with torch.no_grad():
        logits = model(**inputs).logits

    # Decode the predictions to get the phonetic transcription
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)[0]
    
    # Store the result (file name and transcription)
    transcriptions.append([audio_file, transcription])

In [10]:
# Convert the results to a DataFrame and save them to a CSV file
transcriptions_df = pd.DataFrame(transcriptions, columns=['File Name', 'Phonetic Transcription'])
transcriptions_df.to_csv('transcriptions/readingTestFluencE_children.csv', index=False)